#### WITH ... AS
- On its own, AS is a convenient way to clean up the data returned by your query. It's even more powerful when combined with WITH in what's called a "common table expression".

- A common table expression (or CTE) is a temporary table that you return within your query. CTEs are helpful for splitting your queries into readable chunks, and you can write queries against them.

- Example: How many Bitcoin transactions are made per month?

In [ ]:
from google.cloud import bigquery
#Create a "Client" object
client = bigquery.Client()
#Construct a reference to the "crypto_bitcoin" dataset
dataset_ref = client.dataset("crypto_bitcoin", project = "bigquery-public-data")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#Construct a reference to the "transactions" table
table_ref = dataset_ref.table("transactions")
#API request - fetch the table
table = client.get_table(table_ref)
#Preview the first five lines of the "transactions" table
client.list_rows(table, max_results = 5).to_dataframe()

In [ ]:
Using Kaggle's public dataset BigQuery integration.

In [ ]:
# Query to select the number of transactions per date, sorted by date
query_with_CTE = """
                 WITH time AS
                 (
                     SELECT DATE(block_timestamp) AS trans_date
                     FROM `bigquery-public-data.crypto_bitcoin.transactions`
                 )
                 SELECT COUNT(1) AS transactions,
                        trans_date
                 FROM time
                 GROUP BY trans_date
                 ORDER BY trans_date
                 """


In [ ]:
#Set up the query(cancel the query if it would use too much of your
# quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_biled= 10**10)
query_job = client.query(query_with_CTE, job_config = safe_config)
# API request - run the query, and convert the results to a pandas DataFrame
transactions_by_date = query_job.to_dataframe()
#Print the first five rows
transactions_by_date.head()

In [ ]:
#Since they're returned sorted, we can easily plot the raw results to 
#show us the number of Bitcoin transactions per day over the whole timespan
#of this dataset.

In [ ]:
transactions_by_date.set_index('trans_date').plot()

In [ ]:
#As you can see, common table expressions (CTEs) 
#let you shift a lot of your data cleaning into SQL. 
#That's an especially good thing in the case of BigQuery, 
#because it is vastly faster than doing the work in Pandas.